# Exercises

## 1. Create Cone Beam and Helical Geometries

Cone beam and helical geometries (divergent beam geometries) require more parameters for their full characterization.

### Tasks

1. Use the convenience factories to generate all kinds divergent beam geometries.
2. Find out which functions are available to query geometry properties. Which one yields points on the "data manifold", i.e., locations on the detector during rotation?
3. Create a cone beam geometry whose initial source-to-detector vector is $(-4, 0, 0)$ and whose initial detector axes are $(0, 0, -1)$ and $(0, 1, 0)$. What is the handedness of this set of vectors compared to the default one?
4. Achieve the same as in 3. with a transformation matrix and the `frommatrix` constructor.
5. (Advanced) In a 3D cone beam geometry, make use of the vectorized query functions to generate a `200 \times 200 \times 200` volumetric map of distances to the detector for a single source position.

### Good to Know

* `dir(obj)` returns a list of attributes of `obj`.
* A vector of all points in a volume can be generated with the `space.points()` function.
* More information on NumPy broadcasting can be found in [the corresponding NumPy documentation](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html).

## 2. Tomography with TV Regularization

We want to solve the TV-regularized tomography problem

$$
    \min_{x \geq 0} \left[ \| A x - y \|_2^2 + \alpha \| \nabla x \|_1 \right],
$$

where $A: X \to Y$ is the ray transform. You may pick your favorite acquisition geometry (2D, 3D, parallel beam, cone beam, ...). Play with different solvers, regularizers and data fit terms (see e.g. Exercises 2 and 3).

### Tasks

* Solve the TV-regularized tomography problem.
* Use at least one non-smooth and one smooth solver (for the latter, see part 1, exercise 2).
* Add Poisson noise instead of white noise, and use the KL divergence for regularization (see part 1, exercise 3).
* Try TV regularization on the "square trajectory" problem.

### Good to Know

- The data fit $\|\cdot -y\|_2^2$ is now defined on a different space than for denoising, and also $y$ is no longer an element of the reconstruction space $X$. Make sure you understand the details.

## 3. Kaczmarz-type methods (aka ART / SIRT / SART)

Another big class of reconstruction methods is the class of (implicit) least-squares solvers. At convergence, they all solve

$$
    \min_x \|A x - y\|_2^2,
$$

using very different strategies. Prominent examples are [Landweber's method](https://en.wikipedia.org/wiki/Landweber_iteration) (which is nothing but steepest descent for the least-squares problem) and the [conjugate gradient method](https://en.wikipedia.org/wiki/Conjugate_gradient_method).

Kaczmarz-type methods use a strategy of splitting the full problem into blocks of equations, where the splitting is usually done in a way that makes intuitive sense in the context of the problem at hand. For instance, in tomography, single projections or several projections are taken as equation blocks.
In general, the original problem $A x = y$ is rewritten as

$$
    \begin{align}
        A_1 x &= y_1, \\
              &\vdots \\
        A_n x &= y_n
    \end{align}
$$

or more compactly

$$
    \mathbf{A} x = \mathbf{y}, \quad
    \mathbf{A} =
    \begin{pmatrix}
        A_1    \\
        \vdots \\
        A_n
    \end{pmatrix},
    \quad
    \mathbf{y} =
    \begin{pmatrix}
        y_1    \\
        \vdots \\
        y_n
    \end{pmatrix}.
$$

The Kaczmarz method then successively updates $x$ using one block at a time, and repeating the outer loop a given number of times. (The [Wikipedia page](https://en.wikipedia.org/wiki/Kaczmarz_method) explains the basic form of the algorithm.)

### Tasks
- Split the tomography problem into chunks along the angle axis.
- Solve the split problem using the [`odl.solvers.kaczmarz`](https://github.com/odlgroup/odl/blob/ad32a286b69f34260d4428d7282b4058ed2e2603/odl/solvers/iterative/iterative.py#L387-L517) solver.
- Use a different block scheme (block-sequential, interlaced, maximizing inter-block angle distance etc.) for arranging the angles, and investigate the convergence behavior.

Use  to solve a tomography problem by splitting along the angles. Experiment a bit with blocking schemes .

### Good to know

- Least-squares methods regularize by *early termination*, i.e., the methods are stopped after $N$ iterations, and this parameter acts as a regularization parameter (larger $N$ means less regularization).
- ODL has convenience functionality to perform the splitting. Given a `Geometry` object for the full problem, you can use indexing like `geometry[i:i+5]` or `geometry[1::2]` to extract sub-geometries.
- SIRT and SART are Kaczmarz's method in spaces with particular weightings.
- To enforce simple constraints like positivity, a projection onto the desired set can be done in each iteration. This runs under the fancy name "Projection onto Convex Sets" (POCS), but usually just means to set all negative values to zero.